# PostGIS

The city of San Francisco wants to assess the quality of restaurants in the city. Their data is scattered across multiple sources and incomplete.

They tasked you to help them assess it.

They would like to know what the most common violations are where they happen most frequently.

We will use a PostgreSQL database configured with the geo-spatial tool, PostGIS, to do this work.

# Three Database Concepts

1. CRUD
2. connection and cursor
3. transactions

## CRUD

| | SQL | RESTful API |
|:-:|:-:|:-:|
| create | `INSERT` | `POST` |
| read | `SELECT` | `GET` |
| update | `UPDATE` | `PUT` |
| delete | `DELETE` | `DELETE` |


### Correspondence to Python Classes

In [ ]:
class Foo():
    def __init__(self, bar, baz):
        self.bar = int(bar)
        self.baz = int(baz)

This class would correspond to a SQL table.

           foo
    +-------+-------+               
    |  bar  |  baz  |
    +-------+-------+
    |  int  |  int  |
    +-------+-------+    

## Connection and Cursor

We will use a connection object to define our connection to our database over a network.

We will use a cursor object to write to the database. The cursor is created by a class function of the connection object.

In [ ]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

connection = pg2.connect(host='postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)

The cursor can be used to execute queries and then to fetch the results of the query.

In [ ]:
result = cursor.execute("SELECT * FROM business LIMIT 2;")

In [ ]:
cursor.fetchall()

In [ ]:
connection.close()

Because we used the `cursor_factory=RealDictCursor` argument, the results return a list of dictionary objects from the database. 

The advantage of this is that we can display these results easily with a DataFrame.


In [ ]:
import pandas as pd

In [ ]:
connection = pg2.connect(host='postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

In [ ]:
connection.close()

## `lib.db_helper`

Because we will keep using the connection-cursor pattern, we have written it into a sub-module.

In [ ]:
from os import chdir
chdir('/home/jovyan')

In [ ]:
import lib.db_helper as db

#### `db.connect_to_db()`

Contains our connection credentials and returns a connection and cursor. 

    def connect_to_db():
        con = pg2.connect(host='postgis',
                          dbname='postgres',
                          user='postgres')
        cur = con.cursor(cursor_factory=RealDictCursor)
        return con, cur

In [ ]:
connection, cursor = db.connect_to_db()
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

In [ ]:
connection.close()

#### `db.query_to_dictionary()`

1. creates a connection and a cursor
1. uses the cursor to execute a query
1. if `fetch_res` is `True` it fetches the results, otherwise results are `None`
1. closes the connection and returns `results`


    def query_to_dictionary(query, fetch_res=True):
        con, cur = connect_to_db()
        cur.execute(query)
        if fetch_res:
            results = cur.fetchall()
        else:
            results = None
        con.close()
        return results

In [ ]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""")

In [ ]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""", fetch_res=False)

#### `db.query_to_dataframe()`

Wraps `query_to_dictionary()` in a `pandas.DataFrame`.

    def query_to_dataframe(query):
        return DataFrame(query_to_dictionary(query))

In [ ]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

## Transactions

As we begin working on "production" databases, it becomes critical to maintain the integrity of the data. 

Imagine a website with a database using the following table:

           page
    +-----------+---------------+               
    |  page_id  |  visit_count  |
    +-----------+---------------+               
    |    int    |      int      |
    +-----------+---------------+               

Suppose that a page with `page_id` `15` is visited nearly simultaneously. You could imagine the following sequence of events:

1. The first page visit occurs. 
2. The website requests to increment the `visit_count`.
3. The second page visit occurs.
4. Using a separate proces, the website requests to increment the `visit_count`.
5. The first increment is written to the database
6. The second increment is written to the database

How many visits have been recorded?

In order to ensure that no data is lost, PostgreSQL uses transactions.

- A transaction is begun with the keyword `BEGIN`.
- Changes to the database are made in memory.
- Changes to the database are validated. 
- If valid, the changes are made permanent with the keyword `COMMIT`
- If invalid, the changes are discarded with the keyword `ROLLBACK`


### Create new column in `business` table

To demonstrate how transactions work, we will write a SQL query to create a new column in our database.

In [ ]:
db.query_to_dictionary("""
BEGIN;
ALTER TABLE business ADD COLUMN gpnt_location geometry(POINT,4326);
COMMIT;
""", fetch_res=False)

In [ ]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

We will want to populate this column, but the data is a bit messy as we will see in a moment. 

# Postgres Aggregate Functions

In this lesson, we will use [Postgres Aggregate](https://www.postgresql.org/docs/9.5/static/functions-aggregate.html) functions to explore data for businesses in San Francisco. 

Aggregate functions compute a single result from a set of input values.

Some aggregate functions include:

| Function | Description |
|:-:|:-:|
| `avg(expression)` | the average (arithmetic mean) of all input values |
| `count(*)` | number of input rows |
| `max(expression)` | maximum value of `expression` across all input values |
| `min(expression)` | minimum value of `expression` across all input values |
| `sum(expression)` | sum of `expression` across all input values |

# Clean the Data

We can use `COUNT` to count the number of rows in our `business` table. 

In [ ]:
select_count_business = """
SELECT COUNT(*) FROM business
"""

In [ ]:
db.query_to_dataframe(select_count_business)

We can use `COUNT` with a `GROUP BY` clause to count the number of rows for each zip code. 

In [ ]:
select_count_by_postal_code = """
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
"""

In [ ]:
db.query_to_dataframe(select_count_by_postal_code)

## Yikes! That is some messy data. 

### `WHERE` 

We next use the `WHERE` clause to restrict our count to just rows that have `latitude` and `longitude` data. 

In [ ]:
select_count_business_non_null_lat_long = """
SELECT COUNT(*) FROM business
WHERE latitude IS NOT NULL
AND longitude IS NOT NULL
"""

In [ ]:
db.query_to_dataframe(select_count_business_non_null_lat_long)

Note that we might also have invalid data.

In [ ]:
select_count_business_invalid_data = """
SELECT COUNT(*) FROM business
WHERE latitude = 0 AND longitude = 0;"""

In [ ]:
db.query_to_dataframe(select_count_business_invalid_data)

In [ ]:
select_count_business_valid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_count_business_invalid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
"""

In [ ]:
db.query_to_dataframe(select_count_business_valid)

In [ ]:
db.query_to_dataframe(select_count_business_invalid)

### Store Valid and Invalid Queries

We will also make use of subqueries. We will define two:

1. a subquery for all businesses with valid lat/long
1. a subquery for all businesses with invalid lat/long

#### Valid Business Query

In [ ]:
select_business_valid = """
SELECT * FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_business_invalid = """
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
"""

In [ ]:
print(db.query_to_dataframe(select_business_valid).shape)
db.query_to_dataframe(select_business_valid).sample(4)

In [ ]:
print(db.query_to_dataframe(select_business_invalid).shape)
db.query_to_dataframe(select_business_invalid).sample(4)

### Explore Postal Code and Lat/Long

First, let's get the postal codes with a count of less than 10. We will use a sub-query.

Let's use these subqueries to help us to clean the data. 

Note that we re-use `select_count_by_postal_code`.

In [ ]:
print(select_count_by_postal_code)

In [ ]:
select_postal_code_by_postal_code_less_than_10 = """
SELECT postal_code FROM 
    ({}) q
WHERE count < 10
""".format(select_count_by_postal_code)

print(select_postal_code_by_postal_code_less_than_10)

Note that we are using a subquery here. 

e.g. 

    SELECT * FROM (SUBQUERY) sub_query_name;
    

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

We'll think of these as our "bad postal codes". We can think of the query we defined as a list of these codes. That query was

In [ ]:
print(select_postal_code_by_postal_code_less_than_10)

Do we have (lat,long) pairs for these locations? Let's select all rows with one of these postal codes.

### `IN`

To do this we will use the SQL keyword `IN`. `IN` checks a column against a list of items.

e.g. 

    SELECT * FROM my_table WHERE my_column IN ('1,2,3,4,5');

# We will replace the list with our Sub-Query!! 

In [ ]:
select_business_where_bad_postal_code = """
SELECT * FROM business WHERE postal_code IN ({})""".format(select_postal_code_by_postal_code_less_than_10)
print(select_business_where_bad_postal_code)

In [ ]:
db.query_to_dataframe(select_business_where_bad_postal_code).sample(5)

Note that some of these have a lat and long. 

What we want is rows that have "bad postal codes" and no (lat, long) pair. 

### A Nested Query

To get these we will write a nested query that looks like this:

    SELECT * FROM (invalid_lat_long) WHERE postal_code IN (bad_postal_code_str)

In [ ]:
select_invalid_business_where_bad_postal_code = """
SELECT * 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, select_postal_code_by_postal_code_less_than_10)
print(select_invalid_business_where_bad_postal_code)

In [ ]:
db.query_to_dataframe(select_invalid_business_where_bad_postal_code)

#### Let's use `DELETE` to get rid of rows that have no lat, long and a bad zip

First, let's collect a list of business `id`s. 

In [ ]:
select_invalid_business_id_where_bad_postal_code = """
SELECT id 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, select_postal_code_by_postal_code_less_than_10)
print(select_invalid_business_id_where_bad_postal_code)

In [ ]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

In [ ]:
print(select_invalid_business_id_where_bad_postal_code)

We can use this subquery in our `DELETE` query.

Remember, that a **delete** action is a write action and needs to be handled as a transaction. We need to `BEGIN` and `COMMIT`. 

In [ ]:
delete_invalid_business_bad_postal_code = """
BEGIN;
DELETE 
FROM business
WHERE id IN ({});
COMMIT;
""".format(select_invalid_business_id_where_bad_postal_code)
print(delete_invalid_business_bad_postal_code)

Remember, when we execute this we do not need to fetch results so we should set `fetch_res` to `False`.

In [ ]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

Let's have a look at the sql file used to define our database. 

```SQL
CREATE TABLE business (
    id INTEGER,
    name TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code INTEGER,
    latitude FLOAT,
    longitude FLOAT,
    phone_number TEXT,
    PRIMARY KEY (id));

CREATE TABLE inspection (
    business_id INTEGER,
    score INTEGER,
    date TIMESTAMP,
    type TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

CREATE TABLE legend (
    minimum_score INTEGER,
    maximum_score INTEGER,
    description TEXT);

CREATE TABLE violation (
    business_id INTEGER,
    date TIMESTAMP,
    description TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

```

Note that we have foreign key `CONSTRAINT`s on the `inspection` and `violation` tables.  

This means that, in order to drop the businesses with "bad" data, we will need to drop any inspections and violations associated with these. 

### "Bad" Business Data

In [ ]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

In [ ]:
print(select_invalid_business_id_where_bad_postal_code)

In [ ]:
select_violations_for_bad_biz = """
SELECT * FROM violation
WHERE business_id IN ({})
""".format(select_invalid_business_id_where_bad_postal_code)

In [ ]:
db.query_to_dataframe(select_violations_for_bad_biz)

### Delete Violations

In [ ]:
select_count_violation = """
SELECT COUNT(*) FROM violation"""

In [ ]:
db.query_to_dataframe(select_count_violation)

In [ ]:
delete_violations_for_bad_biz = """
BEGIN;
DELETE FROM violation
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [ ]:
db.query_to_dictionary(delete_violations_for_bad_biz, fetch_res=False)

In [ ]:
db.query_to_dataframe(select_count_violation)

### Delete Inspections

In [ ]:
select_count_inspections = """
SELECT COUNT(*) FROM inspection"""

In [ ]:
db.query_to_dataframe(select_count_inspections)

In [ ]:
delete_inspections_for_bad_biz = """
BEGIN;
DELETE FROM inspection
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [ ]:
db.query_to_dictionary(delete_inspections_for_bad_biz, fetch_res=False)

In [ ]:
db.query_to_dataframe(select_count_inspections)

### Delete Businesses

In [ ]:
db.query_to_dataframe(select_count_business)

In [ ]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

In [ ]:
db.query_to_dataframe(select_count_business)

### Pull Bad Postal Codes Again

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

## Repair Bad Zip Codes
To repair the bad zip codes, we will attempt match the nearest points using GIS. 

To do this, we will first need to populate the `gpnt_location` column.

In [ ]:
db.query_to_dataframe(select_business_valid).sample(4)

In [ ]:
update_gpnt = """
BEGIN;
UPDATE business SET 
gpnt_location = ST_SetSRID(ST_MakePoint(latitude, longitude),4326);
COMMIT;
"""

In [ ]:
db.query_to_dictionary(update_gpnt, fetch_res=False)

### Match points to the points with bad zip codes using a given radius

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

In [ ]:
def select_postal_code(postal_code):
    return """SELECT * FROM business WHERE postal_code = {}""".format(postal_code)

In [ ]:
db.query_to_dataframe(select_postal_code(92672))

In [ ]:
def select_gpnt_for_postal_code(postal_code):
    return """SELECT gpnt_location FROM business WHERE postal_code = {} LIMIT 1""".format(postal_code)

In [ ]:
def match_gpnt(postal_code, distance):
    return """
            SELECT *
            FROM business
            WHERE ST_Distance_Sphere(gpnt_location, ({})) <= {}
            """.format(select_gpnt_for_postal_code(postal_code), distance)

In [ ]:
db.query_to_dataframe(match_gpnt(92672, 100))

In [ ]:
update_92672 = """
BEGIN;
UPDATE business SET postal_code = 94117 WHERE postal_code = 92672;
COMMIT;
"""

In [ ]:
db.query_to_dictionary(update_92672, fetch_res=False)

### Check the Status of our Bad Postal Codes

In [ ]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

In [ ]:
db.query_to_dataframe(match_gpnt(94101, 100))

## Plot locations

Next, we will use Folium to plot the businesses for which we have a latitude and a longitude. 

### `folium.Map` 

To build our `folium.Map` object we will need to center the map in a given location. We will use the `avg` aggregate function to do this. 

In [ ]:
import folium

In [ ]:
avg_location_sf = """
SELECT avg(latitude) as avg_lat, 
       avg(longitude) as avg_lon FROM ({}) q;
""".format(select_business_valid)

In [ ]:
db.query_to_dataframe(avg_location_sf)

In [ ]:
avg_loc_df = db.query_to_dataframe(avg_location_sf)

Then assign the result as a single list value.

In [ ]:
avg_loc = avg_loc_df.values.tolist()[0]

In [ ]:
sf_map = folium.Map(location=avg_loc, zoom_start=13)

In [ ]:
within_200_meters_of_94101 = db.query_to_dataframe(match_gpnt(94101, 200))

In [ ]:
near_94101_lat_long = within_200_meters_of_94101[['latitude', 'longitude','postal_code']].values.tolist()

In [ ]:
for loc in near_94101_lat_long:
    try:
        postal_code = str(int(loc[2]))
    except:
        postal_code = None
    folium.Marker(loc[:2],postal_code).add_to(sf_map)

In [ ]:
sf_map

## Maybe that one is OK

# Practice 

See if you can clean up the rest of the "bad" postal codes. If the postal codes are valid, keep them.

# What about those Postal Code `NaN`s?

Train a K Nearest Neighbors Model to Predict the NaN values. 

1. Pull all of the data with a valid lat, long, and postal code.
2. Perform a train test split on the data.
3. Fit a K Nearest Neighbors Model on the training data.
4. Validate the model using the test data. 
5. (optional) Performa Grid Search over the data to find the best number of neighbors to use.
6. (optional) Plot a complexity curve showing your model's accuracy versus the number of neighbors.
7. Pull all of the data with a valid lat, long, but no postal code. 
8. Use your model to assign a postal code to the data. 

# Data Analysis

Consider the `violation` table.


Answer these questions using sql:
1. How many violations are there?
- How many businesses committing violations?
- What's the average number of violations per business?

Answer these questions using python
1. Draw a plot of the violations count
- Is the average number of violations meaningful?
> Not really, the distribution is quite skewed
- draw a plot of the normalized cumulative violation counts. Can we discard the restaurants with few violations?
- where would you draw a threshold if you were to keep 90% of the violations?

Consider the `inspection` table.

Answer these questions using SQL:
1. What's the average score for the whole city?
- What's the average score per business?
- Does the score correlate with the number of inspections?
- Create a dataframe from a table with the following columns:
    business_id, average_score, number_of_inspections, number_of_violations
- Use pandas to do a scatter matrix plot of average_score, number_of_inspections, number_of_violations to check for correlations

### Postal Code Analysis

The town administration would like to know which zip code are the ones where they should focus the inspections.

Use the information contained in the `business` table as well as the previous tables to answer the following questions using SQL:

1. Count the number of businesses per zipcode and sort them by descending order
- Which are the top 5 zipcodes with the worst average score?
    - Restrict your analysis to the zipcodes with at least 50 businesses
    - Do a simple average of the inspections scores in the postal code
- Which are the top 5 zipcodes with the highest number of violations per restaurant?
    - Restrict your  analysis to the zipcodes with at least 50 businesses



## Final recommendation
Give a final recommendation on which 2 zipcodes should the administration focus and choose an appropriate plot to 